### RateBeer.com Scrape Scrape Scrape 

In [1]:
import requests
import sqlalchemy
import numpy as np 
import urllib.request
from bs4 import BeautifulSoup
import re
import csv
import unicodedata

In [2]:
response = requests.get('https://www.ratebeer.com/beer-ratings/')

In [3]:
# response.content #test it on the content

In [4]:
#lets get text instead of bytes
# response.text 

In [5]:
response.headers

{'Date': 'Thu, 06 Feb 2020 19:51:31 GMT', 'Content-Type': 'text/html;charset=UTF-8,text/html;charset=UTF-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Set-Cookie': '__cfduid=dcb066369904234f0c0aacd9b8c22dace1581018691; expires=Sat, 07-Mar-20 19:51:31 GMT; path=/; domain=.ratebeer.com; HttpOnly; SameSite=Lax; Secure, legacy_session=SCDQTSDQ/NIGKIMACAINDGJDECHCGKEAI; path=/; HttpOnly', 'Cache-Control': 'private', 'X-Powered-By': 'ASP.NET', 'X-Frame-Options': 'SAMEORIGIN', 'X-XSS-Protection': '1; mode=block', 'CF-Cache-Status': 'DYNAMIC', 'Expect-CT': 'max-age=604800, report-uri="https://report-uri.cloudflare.com/cdn-cgi/beacon/expect-ct"', 'Server': 'cloudflare', 'CF-RAY': '560fa8c4daf5f055-EWR', 'Content-Encoding': 'gzip'}

In [6]:
response.headers['Content-Type']

'text/html;charset=UTF-8,text/html;charset=UTF-8'

In [7]:
# url = response.request.url

### Make it prettier to read

<a id='Pretty_Tree'></a>

In [8]:
URL = 'https://www.ratebeer.com/beer-ratings/'
r = requests.get(URL) 
  
soup = BeautifulSoup(r.content, 'html5lib') 
print(soup.prettify())

<html>
 <head>
  <meta charset="utf-8" content="text/html;" http-equiv="Content-Type"/>
  <title>
   Beer Reviews - The latest beer reviews - Most Recent
  </title>
  <meta content="The latest beer reviews from RateBeer, the largest collection of beer reviews in the world." name="description"/>
  <meta content="beer, beer reviews, ratebeer, craft beer" name="keywords"/>
  <link href="/css/inc_bubbles.css" rel="stylesheet" type="text/css"/>
  <link href="/tags/tags.css" rel="stylesheet" type="text/css"/>
  <link href="/css/jquery-ui-1.9.2.custom.css" rel="stylesheet" type="text/css"/>
  <link href="/css/brewer_tooltip.css" rel="stylesheet" type="text/css"/>
  <!-- Optimize displays variants to your website visitors
by modifying the DOM (Document Object Model).
In some instances, changes are made to elements that
are already visible to the end user.
These visible changes are referred to as page flicker.

One of the most effective ways to mitigate the risk
of page flicker when loading Opt

### Let's inspect the page - some snips 

![inspect1](Images/inspect1.PNG)

![inspect2](Images/inspectcontinued.PNG)

## Get list of urls for tabs
#### there are 6 tabs that allow access without registration: Recent, New, Faves, Raves, Rants, Anomalous
#### while the tab for My Local doesn't allow access, simply typing a '7' after the url gives full access
#### the max pages are accurate as of 2/6/20

### Recent Tab

In [9]:
# https://www.ratebeer.com/beer-ratings/0/1/       #### 1st page 
# https://www.ratebeer.com/beer-ratings/0/2/       #### 2nd page 
# https://www.ratebeer.com/beer-ratings/0/100/     #### last page 

### New

In [10]:
# https://www.ratebeer.com/beer-ratings/5/1/      #### 1st page 
# https://www.ratebeer.com/beer-ratings/5/2/      #### 2nd page
# https://www.ratebeer.com/beer-ratings/5/85/     #### last

### Faves

In [11]:
# https://www.ratebeer.com/beer-ratings/4/1/       #### 1st page 
# https://www.ratebeer.com/beer-ratings/4/2/       #### 2nd page 
# https://www.ratebeer.com/beer-ratings/4/100/     #### last page 

### Raves 

In [12]:
# https://www.ratebeer.com/beer-ratings/1/1/        #### 1st page
# https://www.ratebeer.com/beer-ratings/1/2/        #### 2nd page
# https://www.ratebeer.com/beer-ratings/1/100/      #### max page 

### Rants 

In [13]:
# https://www.ratebeer.com/beer-ratings/2/1/       #### 1st page
# https://www.ratebeer.com/beer-ratings/2/2/       #### 2nd page
# https://www.ratebeer.com/beer-ratings/2/100/     #### max page 

### Anomalous

In [14]:
# https://www.ratebeer.com/beer-ratings/3/1/        #### 1st page
# https://www.ratebeer.com/beer-ratings/3/2/        #### 2nd page
# https://www.ratebeer.com/beer-ratings/3/100/     #### max page 

In [15]:
# https://www.ratebeer.com/beer-ratings/6/       #### requires log in 

### My Local 

In [16]:
# https://www.ratebeer.com/beer-ratings/7/1/       #### 1st page
# https://www.ratebeer.com/beer-ratings/7/2/      #### 2nd page
# https://www.ratebeer.com/beer-ratings/7/100/    #### max page

## Now we need to consider the urls in the scraper so we are not using the wrong urls 

### Recent Urls 

In [17]:
recent_url = 'http://www.ratebeer.com/beer-ratings/0/' # 'most recent' review page
count = 1
while count < 5:                    #=|>>>>>>>>>>  change the 'count < 5' to increase scrape up to 100 pages
    new_url = (f'{recent_url}{count}/')
    count +=1
    print(new_url) 

http://www.ratebeer.com/beer-ratings/0/1/
http://www.ratebeer.com/beer-ratings/0/2/
http://www.ratebeer.com/beer-ratings/0/3/
http://www.ratebeer.com/beer-ratings/0/4/


In [18]:
# fixed 

### Beer Name Scrape from Recent Tab

In [19]:
# set 'count < 100' to return all pages 
beers = []
recent_url = 'http://www.ratebeer.com/beer-ratings/0/' # 'most recent' review page
count = 1
while count < 5:                    #=|>>>>>>>>>>  change the 'count < 5' to increase scrape up to 100 pages (returns 1st 4)
    new_url = (f'{recent_url}{count}/')
    count +=1
    print(new_url) #comment out if you dont want to see all the lines printed

    page = requests.get(new_url, verify=True)
    page.status_code

    soup = BeautifulSoup(page.text, 'html.parser')
    
    name_soup = soup.findAll('a', {'style': 'font-size:20px; font-weight:bold;'})
      
    for beer_name in name_soup:
        print(beer_name.text)  
        beers.append(beer_name.text)
       

http://www.ratebeer.com/beer-ratings/0/1/
Kirkstall Dexter - Salted Caramel
Hopfenhäcker Isle of Islay Rum Smokey Sten
Hopfenkopf s'Weibal 
Hopfenhäcker Barley Wine
Glen Affric Speyside BA Black Brose
Arrogant Enter Night
Waldhaus Ohne Filter Naturtrüb
Schneider Weisse Tap 5 Meine Hopfen-Weisse
Firestone Walker Double Barrel Ale (DBA)
Emsworth Brewhouse Mainsail
DuClaw Sweet Baby Java 
Cigar City Marshal Zhukov's Imperial Stout
Foolproof Vanilla Espresso Raincloud
Titanic Plum Porter
Left Hand Milk Stout-Chai
http://www.ratebeer.com/beer-ratings/0/2/
Deschutes Ginger Cookie Imperial Porter
Deschutes Luna Jo
Sudden Death Thicker Than Blood
Deschutes Rauch O Marx
Deschutes Bachelor Bitter
Wylam / Old Chimneys Summon Up The Blood
Deschutes Almond Oat Sour
Glen Affric Nutty About You
Deschutes Earl Grey IPA
Young's Cityscape
Half Acre Original Reaper
Stone Go To IPA
Pressure Drop / Left Handed Giant Escape Pod
Orpheus Searching for Flowers on Well-Worn Paths
Lemonstreet Zitronenstraße
http

In [20]:
print(beers) #now we have a list of all the names of the beer from recents tab

['Kirkstall Dexter - Salted Caramel', 'Hopfenhäcker Isle of Islay Rum Smokey Sten', "Hopfenkopf s'Weibal ", 'Hopfenhäcker Barley Wine', 'Glen Affric Speyside BA Black Brose', 'Arrogant Enter Night', 'Waldhaus Ohne Filter Naturtrüb', 'Schneider Weisse Tap 5 Meine Hopfen-Weisse', 'Firestone Walker Double Barrel Ale (DBA)', 'Emsworth Brewhouse Mainsail', 'DuClaw Sweet Baby Java ', "Cigar City Marshal Zhukov's Imperial Stout", 'Foolproof Vanilla Espresso Raincloud', 'Titanic Plum Porter', 'Left Hand Milk Stout-Chai', 'Deschutes Ginger Cookie Imperial Porter', 'Deschutes Luna Jo', 'Sudden Death Thicker Than Blood', 'Deschutes Rauch O Marx', 'Deschutes Bachelor Bitter', 'Wylam / Old Chimneys Summon Up The Blood', 'Deschutes Almond Oat Sour', 'Glen Affric Nutty About You', 'Deschutes Earl Grey IPA', "Young's Cityscape", 'Half Acre Original Reaper', 'Stone Go To IPA', 'Pressure Drop / Left Handed Giant Escape Pod', 'Orpheus Searching for Flowers on Well-Worn Paths', 'Lemonstreet Zitronenstraße

In [21]:
# ratings = soup.findAll('span', attrs={"class": "uas"})
# ratings
# for rating in ratings:
#     print(rating.text)

In [22]:
# from 'recent'
# beer_ratings = []
# url = 'http://www.ratebeer.com/beer-ratings/'
# count = 0
# while count < 2:
#     new_url = (f'{url}{count}/')
#     count +=1
#     print(new_url)

#     page = requests.get(new_url, verify=True)
#     page.status_code

#     soup = BeautifulSoup(page.text, 'html.parser')
    
#     rating_soup = soup.findAll('span', attrs={'class':'uas'})
# #       
#     for beer_rating in rating_soup:
#         print(beer_rating.text)
#         beer_ratings.append(beer_rating.text)

In [23]:
# print(beer_ratings)

In [24]:
# we need to get ride of that '\xa0'
# beer_ratings.remove('\xa0')

In [25]:
# print(beer_ratings) #there are more than 1 so we got rid of the first value only 

#### fixed

In [26]:
# lets try changing loop to this: 
# for beer_rating in rating_soup:
#     if beer_rating.text == '\xa0':
#         pass
#     else:
#         print(beer_rating.text)
#         beer_ratings.append(beer_rating.text)

### Rating Scrape from Recent tab

#### Navigate to Pretty Tree:
[Pretty Tree](#Pretty_Tree)

In [27]:
# from 'most recent' reviews page
beer_ratings = []
recent_url = 'http://www.ratebeer.com/beer-ratings/0/'
count = 1
while count < 5:                  #=|>>>>>>>>>>  change the 'count < 5' to increase scrape up to 100 pages
    new_url = (f'{recent_url}{count}/')
    count +=1
    print(new_url)

    page = requests.get(new_url, verify=True)
    page.status_code

    soup = BeautifulSoup(page.text, 'html.parser')
    
    rating_soup = soup.findAll('span', attrs={'class':'uas'})
    
    for beer_rating in rating_soup:
        if beer_rating.text == '\xa0':
            pass
        else:
            print(beer_rating.text)
            beer_ratings.append(beer_rating.text)

http://www.ratebeer.com/beer-ratings/0/1/
3.4
3.6
3.1
3.5
3.6
3.6
3.6
3.7
3.9
3.2
3.4
3.9
3.0
3.6
3.6
http://www.ratebeer.com/beer-ratings/0/2/
3.7
3.6
3.6
3.3
3.5
4.2
3.2
3.3
3.4
3.4
3.7
3.6
4.3
3.4
3.6
http://www.ratebeer.com/beer-ratings/0/3/
2.6
3.1
4.4
3.5
3.7
3.0
3.6
3.5
3.7
4.1
3.5
3.8
3.5
3.6
3.9
http://www.ratebeer.com/beer-ratings/0/4/
3.2
3.7
3.0
3.5
3.6
3.4
3.8
3.9
4.0
3.6
2.5
3.5
4.1
3.2
3.6


In [28]:
print(beer_ratings) 

['3.4', '3.6', '3.1', '3.5', '3.6', '3.6', '3.6', '3.7', '3.9', '3.2', '3.4', '3.9', '3.0', '3.6', '3.6', '3.7', '3.6', '3.6', '3.3', '3.5', '4.2', '3.2', '3.3', '3.4', '3.4', '3.7', '3.6', '4.3', '3.4', '3.6', '2.6', '3.1', '4.4', '3.5', '3.7', '3.0', '3.6', '3.5', '3.7', '4.1', '3.5', '3.8', '3.5', '3.6', '3.9', '3.2', '3.7', '3.0', '3.5', '3.6', '3.4', '3.8', '3.9', '4.0', '3.6', '2.5', '3.5', '4.1', '3.2', '3.6']


In [29]:
# now lets put it together
# using naive method 
# to convert lists to dictionary 
# resulting_dict = {} 
# for key in test_keys: 
#     for value in test_values: 
#         res[key] = value 
#         test_values.remove(value) 
#         break  

In [30]:
# Using naive method
name_rating = {}
for key in beers:
    for value in beer_ratings:
        name_rating[key] = value
        beer_ratings.remove(value)
        break

In [31]:
# next time using list comprehension since it looks cooler
# using dictionary comprehension 
# to convert lists to dictionary 
# res = {test_keys[i]: test_values[i] for i in range(len(test_keys))}

In [32]:
print(name_rating)

{'Kirkstall Dexter - Salted Caramel': '3.4', 'Hopfenhäcker Isle of Islay Rum Smokey Sten': '3.6', "Hopfenkopf s'Weibal ": '3.1', 'Hopfenhäcker Barley Wine': '3.5', 'Glen Affric Speyside BA Black Brose': '3.6', 'Arrogant Enter Night': '3.6', 'Waldhaus Ohne Filter Naturtrüb': '3.6', 'Schneider Weisse Tap 5 Meine Hopfen-Weisse': '3.7', 'Firestone Walker Double Barrel Ale (DBA)': '3.9', 'Emsworth Brewhouse Mainsail': '3.2', 'DuClaw Sweet Baby Java ': '3.4', "Cigar City Marshal Zhukov's Imperial Stout": '3.9', 'Foolproof Vanilla Espresso Raincloud': '3.0', 'Titanic Plum Porter': '3.6', 'Left Hand Milk Stout-Chai': '3.6', 'Deschutes Ginger Cookie Imperial Porter': '3.7', 'Deschutes Luna Jo': '3.6', 'Sudden Death Thicker Than Blood': '3.6', 'Deschutes Rauch O Marx': '3.3', 'Deschutes Bachelor Bitter': '3.5', 'Wylam / Old Chimneys Summon Up The Blood': '4.2', 'Deschutes Almond Oat Sour': '3.2', 'Glen Affric Nutty About You': '3.3', 'Deschutes Earl Grey IPA': '3.4', "Young's Cityscape": '3.4', 

<div class="alert alert-info">
<strong>Note: The items are accurate per the links! If you want to get the max pages, just change BOTH scrape loops from 'count < 5' to 'count < 100' 
</div>

### Simply change the soup recipie for the loop and get the review text, location ... and update list 

### Get text reviews scrape from Recent tab

In [33]:
#  add a teaspoon of text 
# breweries_rating_texts = soup.findAll('div', {'style': 'color:#666;'})

In [34]:
# from 'most recent' reviews page
reviews = []
recent_url = 'http://www.ratebeer.com/beer-ratings/0/'
count = 1
while count < 5:                  #=|>>>>>>>>>>  change the 'count < 5' to increase scrape up to 100 pages
    new_url = (f'{recent_url}{count}/')
    count +=1
    print(new_url)

    page = requests.get(new_url, verify=True)
    page.status_code

    soup = BeautifulSoup(page.text, 'html.parser')
    
    review_soup = soup.findAll('div', {'style':'color:#666;'})
    
    for review in review_soup:
            print(review.text)
            reviews.append(review.text)

http://www.ratebeer.com/beer-ratings/0/1/
Keg at the Beer Pharmacie, Syston. Dark brown with a ring of khaki/tan head. Milk chocokate, a tpuch of cola nut in the aroma, with less of the advertised salted caramel than I expected although it's still a component. Tastes sweet with toffee notes to the fore, followed by a light hop bitterness in the finish. Liquorice in the aftertaste along with more chocolate and caramel. Light carbonation. While not the most focused beer, it certainly has a pleasant composition.
Sampled @ Braukunst Live 2020. A very dark brown beer with a beige head. Aroma of bourbon, rum Barrel, dark roasted malt. Taste of dark roasted malt, coffee, burned malt rum barrel.
Sampled @ Braukunst Live 2020. A slightly hazy yellow golden beer with a white head. Aroma of sweet banana, quite yeasty. Taste of sweet banana, yeast, some phenols, quite sweet.
Sampled @ Braukunst Live 2020. A slightly hazy dark amber beer with a off white head. Aroma of strong caramelized malt, red 

Cask at MS HW, 06/02/20.
Clear golden blonde with a decent white head.
Nose is light coffee bean, airy malts, straw, vanilla, toasted malts.
Taste comprises creamy malts, faint coffee, hint of white chocolate buttons, vanilla, light fruit esters.
Medium bodied, soft carbonation, semi drying close.
Passable for the sub genre ... nothing I’d go back to though.
Top notch sour, balanced and drinkable. Fruity yet not overly so, and tartness that is world class. GBBF Winter
Im Geruch malzig süßlich. Goldgelbes klares Bier, schnell einfallende grobe Schaumkrone. Der Antrunk ist süßlich hopfig-malzig. Traditioneller Hopfen im Aroma. Sehr mild, kaum bitter. Leichte Säure. Feinperlig. Relativ eintönig, leistet sich jedoch keine geschmacklichen Fehler. Besser als das typische Durchschnitts-Export.
Clear golden with a thin head.
Grainy, yeasty smell with a citric note. Grainy taste. A normal Belgian lager/pils as there are many. Highly drinkable.
Meantime Salted Caramel Porter 5.5%. Chestnut brown

In [35]:
print(reviews)

["Keg at the Beer Pharmacie, Syston. Dark brown with a ring of khaki/tan head. Milk chocokate, a tpuch of cola nut in the aroma, with less of the advertised salted caramel than I expected although it's still a component. Tastes sweet with toffee notes to the fore, followed by a light hop bitterness in the finish. Liquorice in the aftertaste along with more chocolate and caramel. Light carbonation. While not the most focused beer, it certainly has a pleasant composition.", 'Sampled @ Braukunst Live 2020. A very dark brown beer with a beige head. Aroma of bourbon, rum Barrel, dark roasted malt. Taste of dark roasted malt, coffee, burned malt rum barrel.', 'Sampled @ Braukunst Live 2020. A slightly hazy yellow golden beer with a white head. Aroma of sweet banana, quite yeasty. Taste of sweet banana, yeast, some phenols, quite sweet.', 'Sampled @ Braukunst Live 2020. A slightly hazy dark amber beer with a off white head. Aroma of strong caramelized malt, red malt, caramel, honey. Taste of 

### Get location scrape from Recent 

In [36]:
# soup.findAll('span')
# locations = soup.findAll('span', {'class': 'small'})

In [37]:
# from 'most recent' reviews page
locations = []
recent_url = 'http://www.ratebeer.com/beer-ratings/0/'
count = 1
while count < 5:                  #=|>>>>>>>>>>  change the 'count < 5' to increase scrape up to 100 pages
    new_url = (f'{recent_url}{count}/')
    count +=1
    print(new_url)

    page = requests.get(new_url, verify=True)
    page.status_code

    soup = BeautifulSoup(page.text, 'html.parser')
    
    location_soup = soup.findAll('span', {'class': 'small'})
    
    for location in location_soup:
            print(location.text)
            locations.append(location.text)

http://www.ratebeer.com/beer-ratings/0/1/
Leeds, West Yorkshire                   
München, Bavaria                          
Feichten an der Alz, Bavaria                          
München, Bavaria                          
Birkenhead, Merseyside                       
Escondido, California                       
Waldhaus, Baden-Württemberg                
Kelheim, Bavaria                          
Paso Robles, California                       
Havant, Hampshire                        
Baltimore, Maryland                         
Tampa, Florida                          
Pawtucket, Rhode Island                     
Stoke-on-Trent, Staffordshire                    
Longmont, Colorado                         
http://www.ratebeer.com/beer-ratings/0/2/
Bend, Oregon                           
Bend, Oregon                           
Timmendorfer Strand, Schleswig-Holstein               
Bend, Oregon                           
Bend, Oregon                           
Newcastle upon Tyne , Tyne 

In [38]:
print(locations)

['Leeds, West Yorkshire                   ', 'München, Bavaria                          ', 'Feichten an der Alz, Bavaria                          ', 'München, Bavaria                          ', 'Birkenhead, Merseyside                       ', 'Escondido, California                       ', 'Waldhaus, Baden-Württemberg                ', 'Kelheim, Bavaria                          ', 'Paso Robles, California                       ', 'Havant, Hampshire                        ', 'Baltimore, Maryland                         ', 'Tampa, Florida                          ', 'Pawtucket, Rhode Island                     ', 'Stoke-on-Trent, Staffordshire                    ', 'Longmont, Colorado                         ', 'Bend, Oregon                           ', 'Bend, Oregon                           ', 'Timmendorfer Strand, Schleswig-Holstein               ', 'Bend, Oregon                           ', 'Bend, Oregon                           ', 'Newcastle upon Tyne , Tyne & Wear                

### Get user info from Recent 

#### Navigate to Pretty Tree:
[Pretty Tree](#Pretty_Tree)

In [39]:
# user_soup = something like this where loubar is the user_name and 334082 is the user_id

# <a class="small" href="/user/334082/">
#             <span style="color:#8b8b8b;">            <<<<<<< lets try this 
#              loubar
#             </span>
#            </a>
#            <a href="/user/334082/">

In [40]:
# # from 'most recent' reviews page
# users = []
# recent_url = 'http://www.ratebeer.com/beer-ratings/0/'
# count = 1
# while count < 5:                  #=|>>>>>>>>>>  change the 'count < 5' to increase scrape up to 100 pages
#     new_url = (f'{recent_url}{count}/')
#     count +=1
#     print(new_url)

#     page = requests.get(new_url, verify=True)
#     page.status_code

#     soup = BeautifulSoup(page.text, 'html.parser')
    
#     user_soup = soup.findAll('span', {'style':'color:#8b8b8b;'})
    
#     for user in user_soup:
#             print(user.text)
#             users.append(user.text)

http://www.ratebeer.com/beer-ratings/0/1/
Leeds, West Yorkshire                   
jjsint
München, Bavaria                          
Beerhunter111
Feichten an der Alz, Bavaria                          
Beerhunter111
München, Bavaria                          
Beerhunter111
Birkenhead, Merseyside                       
MarcinG
Escondido, California                       
bergstaden
Waldhaus, Baden-Württemberg                
martjoobolut
Kelheim, Bavaria                          
edward78
Paso Robles, California                       
hrabren
Havant, Hampshire                        
kwik-lime
Baltimore, Maryland                         
Bebum
Tampa, Florida                          
TEJA
Pawtucket, Rhode Island                     
Bebum
Stoke-on-Trent, Staffordshire                    
bdleedahl
Longmont, Colorado                         
tmoreau
http://www.ratebeer.com/beer-ratings/0/2/
Bend, Oregon                           
BeerandBlues2
Bend, Oregon                           
Beera

### retuning the location of the brewery AND then the user who wrote review because they are the same color

In [ ]:
# try skipping every other one or fix the query 

# Slice notation a[start_index:end_index:step]

# return a[::2]
# where start_index defaults to 0 and end_index defaults to the len(a).

In [42]:
# # from 'most recent' reviews page
# users = []
# recent_url = 'http://www.ratebeer.com/beer-ratings/0/'
# count = 1
# while count < 5:                  #=|>>>>>>>>>>  change the 'count < 5' to increase scrape up to 100 pages
#     new_url = (f'{recent_url}{count}/')
#     count +=1
#     print(new_url)

#     page = requests.get(new_url, verify=True)
#     page.status_code

#     soup = BeautifulSoup(page.text, 'html.parser')
    
#     user_soup = soup.findAll('span', {'style':'color:#8b8b8b;'})
    
#     for user in user_soup:
#             print(user.text[::2])
# #             users.append(user.text)

http://www.ratebeer.com/beer-ratings/0/1/
Les etYrsie         
jsn
Mnhn aai             
Behne11
Fihe ndrAz aai             
Behne11
Mnhn aai             
Behne11
Breha,Mresd            
MriG
Ecnio aiona           
brsae
Wlhu,BdnWrtmeg        
mrjoou
Klem aai             
ewr7
Ps ols aiona           
harn
Hvn,Hmsie            
ki-ie
Blioe ayad            
Bbm
Tma lrd             
TJ
Pwukt hd sad          
Bbm
Soeo-rn,Safrsie          
bleal
Lnmn,Clrd             
toeu
http://www.ratebeer.com/beer-ratings/0/2/
Bn,Oeo              
Beadle2
Bn,Oeo              
Beadle2
Tmedre tad clsi-osen       
jegn
Bn,Oeo              
Beadle2
Bn,Oeo              
Beadle2
Nwateuo ye,Tn  er         
Ingd
Bn,Oeo              
Beadle2
Breha,Mresd            
MriG
Bn,Oeo              
Beadle2
Bdod efrsie          
jsn
Ciao lios            
toeu
Ecnio aiona           
rmLS
TtehmHl,GetrLno          
Ingd
Alna eri             
mk_7
Jcsnil,Foia             
jwtsugr
http://www.ratebeer.com/beer-ratings/0/3/
Hrf

In [43]:
# not quite there yet

In [44]:
# # from 'most recent' reviews page
# users = []
# recent_url = 'http://www.ratebeer.com/beer-ratings/0/'
# count = 1
# while count < 5:                  #=|>>>>>>>>>>  change the 'count < 5' to increase scrape up to 100 pages
#     new_url = (f'{recent_url}{count}/')
#     count +=1
#     print(new_url)

#     page = requests.get(new_url, verify=True)
#     page.status_code

#     soup = BeautifulSoup(page.text, 'html.parser')
    
#     user_soup = soup.findAll('span', {'style':'color:#8b8b8b;'})
    
#     for user in user_soup[::2]:
#             print(user.text)
#             users.append(user.text)

http://www.ratebeer.com/beer-ratings/0/1/
Leeds, West Yorkshire                   
München, Bavaria                          
Feichten an der Alz, Bavaria                          
München, Bavaria                          
Birkenhead, Merseyside                       
Escondido, California                       
Waldhaus, Baden-Württemberg                
Kelheim, Bavaria                          
Paso Robles, California                       
Havant, Hampshire                        
Baltimore, Maryland                         
Tampa, Florida                          
Pawtucket, Rhode Island                     
Stoke-on-Trent, Staffordshire                    
Longmont, Colorado                         
http://www.ratebeer.com/beer-ratings/0/2/
Bend, Oregon                           
Bend, Oregon                           
Timmendorfer Strand, Schleswig-Holstein               
Bend, Oregon                           
Bend, Oregon                           
Newcastle upon Tyne , Tyne 

In [45]:
# Getting warmer

### keep trying 
arr = list(range(10)) # Range from 0-9

### List comprehension: Range with conditional
print [arr[index] for index in range(len(arr)) if index % 2 == 0]

### List comprehension: Range with step
print [arr[index] for index in range(0, len(arr), 2)]

### List comprehension: Enumerate with conditional
print [item for index, item in enumerate(arr) if index % 2 == 0]

### List filter: Index in range
print filter(lambda index: index % 2 == 0, range(len(arr)))

### Extended slice
print arr[::2]

In [51]:
# from 'most recent' reviews page
users = []
recent_url = 'http://www.ratebeer.com/beer-ratings/0/'
count = 1
while count < 5:                  #=|>>>>>>>>>>  change the 'count < 5' to increase scrape up to 100 pages
    new_url = (f'{recent_url}{count}/')
    count +=1
    print(new_url)

    page = requests.get(new_url, verify=True)
    page.status_code

    soup = BeautifulSoup(page.text, 'html.parser')
    
    user_soup = soup.findAll('span', {'style':'color:#8b8b8b;'})
    
    for user in user_soup[1::2]:
            print(user.text)
#             users.append(user.text)

http://www.ratebeer.com/beer-ratings/0/1/
Mr_Pink_152
dwyerpg
Leighton
Leighton
dwyerpg
jjsint
Beerhunter111
Beerhunter111
Beerhunter111
MarcinG
bergstaden
martjoobolut
edward78
hrabren
kwik-lime
http://www.ratebeer.com/beer-ratings/0/2/
Bebum
TEJA
Bebum
bdleedahl
tmoreau
BeerandBlues2
BeerandBlues2
joergen
BeerandBlues2
BeerandBlues2
Iznogud
BeerandBlues2
MarcinG
BeerandBlues2
jjsint
http://www.ratebeer.com/beer-ratings/0/3/
tmoreau
remALUS
Iznogud
mike_77
jcwattsrugger
Quick3Beers
joergen
snowcrash000
minihame2
tmoreau
kingstoni
joergen
TBone
MarcinG
PlutonowyMan
http://www.ratebeer.com/beer-ratings/0/4/
Parmenion777
TheWhaleNorman
Beese
tmoreau
loubar
Theydon_Bois
bdleedahl
ichijo3
ophone
MaltyVonHop
Parmenion777
wojanex
prestonrail
JorisPPattyn
joergen


In [52]:
# fixed 

### Recap: We now have the ability to scrape user names, review, ratings, beer names and location of brewery
#### It might not have been pretty, but it works

### Once funtion is defined, we can easily replace the url for the other tabs, such as favs, raves, new, ... 

## Let's put it all together now

In [89]:
# Combined scrape from 'most recent' reviews page

beers = []
ratings = []
locations = []
reviews = []
users = []

recent_url = 'http://www.ratebeer.com/beer-ratings/0/'

count = 1
while count < 2:                  #=|>>>>>>>>>>  change the 'count < x' to increase scrape up to 100 pages
    new_url = (f'{recent_url}{count}/')
    count +=1
    print(new_url)

    page = requests.get(new_url, verify=True)
    page.status_code

#   make a nice soup 
    soup = BeautifulSoup(page.text, 'html.parser')
    
    print("Beer Names: ")
                  
#   beer names scrape        
    name_soup = soup.findAll('a', {'style': 'font-size:20px; font-weight:bold;'})
      
    for beer_name in name_soup:
        print(beer_name.text)  
        beers.append(beer_name.text)
        
    print("Beer Ratings: ")
        
#   ratings scrape
    rating_soup = soup.findAll('span', attrs={'class':'uas'})
    
    for beer_rating in rating_soup:
        if beer_rating.text == '\xa0':
            pass
        else:
            print(beer_rating.text)
            ratings.append(beer_rating.text)
    
    print("Brewery Locations: ")
#   location scrape
    location_soup = soup.findAll('span', {'class': 'small'})
    
    for location in location_soup:
            print(location.text)
            locations.append(location.text)
            
    print("User Reviews: ")
    
#   review scrape
    review_soup = soup.findAll('div', {'style':'color:#666;'})
    
    for review in review_soup:
            print(review.text)
            reviews.append(review.text)
            
    print("User Names: ")
    
#   user scrape
    user_soup = soup.findAll('span', {'style':'color:#8b8b8b;'})
    
    for user in user_soup[1::2]:
            print(user.text)
            users.append(user.text)

    

http://www.ratebeer.com/beer-ratings/0/1/
Beer Names: 
Northern Monk / HOMES / Ology Patrons Project 10.07 Culinary Concepts Fruited Sour IPA
Enville Old Porter
Northern Monk Patrons Project 25.01 Everyday Abstract 
Silver Moon Barrel Aged Xul Blend
Silver Moon Raspberry Nights
Berliner Berg It's Been A While IPA
Vista Adair
Silver Moon IPA 97
Evil Twin I Love You With My Stout 
Silver Moon Bangarang!
Woodman's Wild Ale Redruth Red
Silver Moon Moonlight
Emsworth Brewhouse SkIPA
Ruhrtaler India Pale Ale
Gruner Brothers Beulah Brown 
Beer Ratings: 
4.3
3.3
3.8
3.8
3.3
3.5
3.5
3.5
3.9
3.4
3.4
3.1
3.6
3.3
3.3
Brewery Locations: 
Leeds, West Yorkshire                   
Enville, Staffordshire                    
Leeds, West Yorkshire                   
Bend, Oregon                           
Bend, Oregon                           
Neukölln, Berlin, Berlin                           
Driftwood, Texas                            
Bend, Oregon                           
Queens, New York         

# Set the loop to add all lists into a dictionary

In [59]:
### Using dict comprehension to combine to dict
# using dictionary comprehension 
# to convert lists to dictionary 
# res = {test_keys[i]: test_values[i] for i in range(len(test_keys))}

### maybe another time 

In [67]:
# current lists (from the recent tab)
# beers = []
# ratings = []
# locations = []
# reviews = []
# users = []

In [68]:
#  example from earlier
# Using naive method
# name_rating = {}
# for key in beers:
#     for value in beer_ratings:
#         name_rating[key] = value
#         beer_ratings.remove(value)
#         break

In [69]:
# print(beers)

['AND UNION Neu Blk / Neu Black Lager', 'TrimTab Wide World', 'Strangeways Imperial Lucky Leprechaun', 'Brauerei Simon Schwarze Kuni Dunkler Weizenbock', 'Odyssey Just After Midnight', 'Beatnikz Republic I Heart San Diego', 'Gulpener Chateau Neubourg', 'Landgang Weizheit ', 'Bend Primal Dream CDA', 'Siren Going Out Stout', 'Vormann Obernachtwächter Spezial', 'Cloudwater A Scabrous Edge Of The Sky', 'Cigar City Cubano-Style Espresso Brown Ale', 'Bend Trade War Export Stout', 'Fourpure / Thornbridge Treeline']


In [73]:
print(ratings)

['3.3', '3.5', '3.5', '3.5', '3.9', '3.4', '3.4', '3.1', '3.6', '3.3', '3.3', '3.6', '3.4', '3.6', '3.5']


In [83]:
# review_dict = {}
# for key in beers: # remeber this is the beer names
# #     add the beer ratings
#     for value in ratings, locations, reviews, users:
#         review_dict[key] = value
#         ratings.remove(value)
#         locations.remove(value)
#         reviews.remove(value)
#         users.remove(value)
#         break
     

In [85]:
# print(review_dict)

### seek help online 

In [ ]:
# current lists (from the recent tab)
# beers = []
# ratings = []
# locations = []
# reviews = []
# users = []

In [101]:
# review_data = dict(zip(beers, ratings))

In [106]:
# print(review_data)

In [108]:
# review_data = dict(zip(review_data, locations))

In [109]:
# print(review_data)

In [112]:
beer_dict = {}
 
# Add a list:
beer_dict["Beer Name"] = beers
beer_dict["Rating"] = ratings
beer_dict["Location"] = locations
beer_dict["Review"] = reviews
beer_dict["User"] = users

 
# # Add a dictionary:
# my_dict["sub_dict"] = {"key": "value"}
 
# # Add a set:
# my_dict["my_set"] = set([3, 1, 4, 1, 5, 9, 2])
 
# # Add a Tuple as a key and as a value:
# my_dict[(27.9878, 86.9250)] = ("Everest", "Nepal", 8848)
 
# Prove it all worked:
for key, value in beer_dict.items():
  print("Key: {}\nValue: {}\n\n".format(key,value))

Key: Beer Name
Value: ['Northern Monk / HOMES / Ology Patrons Project 10.07 Culinary Concepts Fruited Sour IPA', 'Enville Old Porter', 'Northern Monk Patrons Project 25.01 Everyday Abstract ', 'Silver Moon Barrel Aged Xul Blend', 'Silver Moon Raspberry Nights', "Berliner Berg It's Been A While IPA", 'Vista Adair', 'Silver Moon IPA 97', 'Evil Twin I Love You With My Stout ', 'Silver Moon Bangarang!', "Woodman's Wild Ale Redruth Red", 'Silver Moon Moonlight', 'Emsworth Brewhouse SkIPA', 'Ruhrtaler India Pale Ale', 'Gruner Brothers Beulah Brown ']


Key: Rating
Value: ['4.3', '3.3', '3.8', '3.8', '3.3', '3.5', '3.5', '3.5', '3.9', '3.4', '3.4', '3.1', '3.6', '3.3', '3.3']


Key: Location
Value: ['Leeds, West Yorkshire                   ', 'Enville, Staffordshire                    ', 'Leeds, West Yorkshire                   ', 'Bend, Oregon                           ', 'Bend, Oregon                           ', 'Neukölln, Berlin, Berlin                           ', 'Driftwood, Texas      

### Getting better but not really what we want

### Ideas: turn this into a function that allows user to select url to scrape from input from list of urls: select 1: recent, 2 for favs, 3 for  ...

## or just combine scrape for all urls into one by iterating through list of urls 
### keep in mind the urls all max out at 100 pages except for the "New" tab which goes to 85